In [1]:
from PIL import Image
import requests
import os
import pandas as pd
from transformers import CLIPProcessor, CLIPModel

In [2]:
# Load model directly
from transformers import AutoProcessor, AutoModelForZeroShotImageClassification

processor = AutoProcessor.from_pretrained("patrickjohncyh/fashion-clip")
model = AutoModelForZeroShotImageClassification.from_pretrained("patrickjohncyh/fashion-clip")

In [3]:
outfits= pd.read_csv('./datathon/dataset/outfit_data.csv')
product = pd.read_csv('./datathon/dataset/product_data.csv')

In [9]:
#Deleting all the products that are not clothes
product = product[(product.des_product_category!= 'Beauty') & (product.des_product_category!= 'Home')]
# Delete all products for men and kids due to desbalance
product = product[(product.des_line== 'SHE')]
product = product[(product.des_sex== 'Female')]
product = product[(product.des_age== 'Adult')]
product = product[~product.des_product_aggregated_family.isin(['Dresses and jumpsuits','Swim and intimate'])]
product = product[(product.des_product_aggregated_family!='Accesories') | (product.des_product_family=='Footwear')]
product['des_product_category'] = product['des_product_category'].apply(lambda x: 'Tops' if x =='Outerwear' else x)
product =product.drop(['des_sex','des_line','des_age'], axis = 1)

# Delete the deleted products from outfits
outfits = outfits[outfits.cod_modelo_color.isin(product.cod_modelo_color.unique())]

In [12]:
def getEmbeding(x):
    try:
        img = Image.open(x)
        inputs = processor(images=[img], return_tensors="pt", padding=True)
        return inputs['pixel_values'].view(-1).numpy()
    except:
        return None

product['emb'] = product['des_filename'].apply(getEmbeding)

In [13]:
outfits.to_pickle('./datathon/dataset/final_outfits')
product.to_pickle('./datathon/dataset/final_products')